# Player Data Explorer

This notebook provides interactive visualizations for exploring War Thunder player performance data. Make sure to run the `VehicleDataGrabber` launch option, and then the `ReplayDataGrabber - War Thunder` launch option. Check out the README for more information.

Note that graphs can be explored interactively:
- Zoom: Mouse wheel or zoom tools in toolbar
- Pan: Click and drag
- Filter: Click on legend items to show/hide data
- Reset: Double-click to reset zoom
- Details: Hover over nodes for extra information

## Setup

Set up basic imports and pathing

In [ ]:
# Import standard libraries
import sys
from pathlib import Path
from typing import Any, Optional, Callable

# Import third-party libraries
import pandas as pd
import numpy as np

# Import Plotly and setup offline plotting
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

# Add the project root and src directories to Python path
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent  # Go up one level from src to project root
src_dir = notebook_dir  # Current directory is src

# Add paths to sys.path if they're not already there
for path in [str(project_root), str(src_dir)]:
    if path not in sys.path:
        sys.path.insert(0, path)

print(f"Project root: {project_root}")
print(f"Source directory: {src_dir}")
print(f"Current working directory: {Path.cwd()}")

Import project libraries and initialize services

In [ ]:
# Import project modules
from src.common.configuration import get_config
from src.common.factories import ServiceFactory
from src.replay_data_explorer.services import DataLoaders
from src.replay_data_explorer.services import GraphExporter

# Initialize configuration
config = get_config().replay_data_explorer_config

# Initialize the data loader
service_factory = ServiceFactory()
replay_manager_service = service_factory.get_replay_manager_service()
data_loaders = DataLoaders(replay_manager_service)
graph_exporter = GraphExporter(graph_export_config=config.graph_export_config)

# Import graph creation functions
from src.replay_data_explorer.graphs import *

### Data Loading

Load replay data and process it for analysis.

In [ ]:
player_performance_df = data_loaders.get_player_performance_data(config.player_name, config.country_filters)
global_performance_df = data_loaders.get_global_performance_data(config.country_filters)

### Setup Graph Export

In [ ]:
_exportable_graphs: dict[str, go.Figure] = {}
_exportable_graphs_deferred: dict[str, tuple[Callable[[list, dict], go.Figure], list, dict]] = {}

def add_exportable_graph(graph: go.Figure, graph_name: str):
    _exportable_graphs[graph_name] = graph

def add_exportable_graph_deferred(graph_creator: Callable[[list, dict], go.Figure], *args, **kwargs):
    graph_name = graph_creator.__name__.removeprefix("create_").lower()
    _exportable_graphs_deferred[graph_name] = (graph_creator, args, kwargs)

## Player Performance Analysis

In [ ]:
create_scatter_score_vs_br(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
add_exportable_graph_deferred(create_scatter_score_vs_br, player_performance_df, player_name="Author", country_filters=config.country_filters)

create_bar_score_distribution(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
create_player_score_heatmap_by_country_and_br(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
create_heatmap_br_delta_by_country_and_br(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
create_bar_score_vs_map(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()

## Global Performance Analysis

In [ ]:
all_player_heatmap_score_by_country_and_br = create_all_player_heatmap_score_by_country_and_br(global_performance_df, author_name=config.player_name, country_filters=config.country_filters)
add_exportable_graph(all_player_heatmap_score_by_country_and_br, "all_player_heatmap_score_by_country_and_br")
all_player_heatmap_score_by_country_and_br.show()

all_player_heatmap_premium_score_delta_by_country_and_br = create_all_player_heatmap_premium_score_delta_by_country_and_br(global_performance_df, author_name=config.player_name, country_filters=config.country_filters)
add_exportable_graph(all_player_heatmap_premium_score_delta_by_country_and_br, "all_player_heatmap_premium_score_delta_by_country_and_br")
all_player_heatmap_premium_score_delta_by_country_and_br.show()

all_player_heatmap_br_delta_by_country_and_br = create_all_player_heatmap_br_delta_by_country_and_br(global_performance_df, author_name=config.player_name, country_filters=config.country_filters)
add_exportable_graph(all_player_heatmap_br_delta_by_country_and_br, "all_player_heatmap_br_delta_by_country_and_br")
all_player_heatmap_br_delta_by_country_and_br.show()

all_player_heatmap_premium_br_delta_by_country_and_br = create_all_player_heatmap_premium_br_delta_by_country_and_br(global_performance_df, author_name=config.player_name, country_filters=config.country_filters)
add_exportable_graph(all_player_heatmap_premium_br_delta_by_country_and_br, "all_player_heatmap_premium_br_delta_by_country_and_br")
all_player_heatmap_premium_br_delta_by_country_and_br.show()

## Tier Frequency Analysis

In [ ]:
create_bar_tier_distribution(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
add_exportable_graph_deferred(create_bar_tier_distribution, player_performance_df, player_name="Author", country_filters=config.country_filters)

create_pie_tier_frequency(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
add_exportable_graph_deferred(create_pie_tier_frequency, player_performance_df, player_name="Author", country_filters=config.country_filters)

create_bar_tier_frequency_vs_country(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
add_exportable_graph_deferred(create_bar_tier_frequency_vs_country, player_performance_df, player_name="Author", country_filters=config.country_filters)

create_bar_tier_frequency_vs_br(player_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
add_exportable_graph_deferred(create_bar_tier_frequency_vs_br, player_performance_df, player_name="Author", country_filters=config.country_filters)

## Player Squad Performance

In [ ]:
if config.player_name:
    create_bar_squad_performance(global_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
    add_exportable_graph_deferred(create_bar_squad_performance, global_performance_df, player_name=config.player_name, display_player_name="Author", country_filters=config.country_filters)

    create_bar_squad_win_rate(global_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
    add_exportable_graph_deferred(create_bar_squad_win_rate, global_performance_df, player_name=config.player_name, display_player_name="Author", country_filters=config.country_filters)

    create_bar_squad_tier_distribution(global_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
    add_exportable_graph_deferred(create_bar_squad_tier_distribution, global_performance_df, player_name=config.player_name, display_player_name="Author", country_filters=config.country_filters)

    create_bar_squad_br_delta(global_performance_df, player_name=config.player_name, country_filters=config.country_filters).show()
    add_exportable_graph_deferred(create_bar_squad_br_delta, global_performance_df, player_name=config.player_name, display_player_name="Author", country_filters=config.country_filters)
else:
    print("Skipping squad performance graphs as no player name is configured.")

## Perform Graph Export

In [ ]:
for graph_name, (graph_creator, args, kwargs) in _exportable_graphs_deferred.items():
    graph_figure = graph_creator(*args, **kwargs)
    print(f"Exporting deferred graph: {graph_name}")
    graph_exporter.save_as_png(graph_figure, graph_name)

for graph_name, graph_figure in _exportable_graphs.items():
    print(f"Exporting graph: {graph_name}")
    graph_exporter.save_as_png(graph_figure, graph_name)